###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [3]:
import pandas as pd
import sqlite3
from pathlib import Path
from google.colab import drive

In [ ]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .

In [4]:
# Finding path
drive.mount('/content/drive')
!ls drive/MyDrive/cs/cs133/Assignments

Mounted at /content/drive
A2_intro-colab.ipynb	   A6_seaborn.ipynb		     Colab_A12_HO12_sqlite.ipynb
A3_pandas.ipynb		   A7_scatter.ipynb		     Colab_A8_plot_Marvel.ipynb
A4_data_exploration.ipynb  Colab_A10_complicated-data.ipynb  Colab_A9_transform.ipynb
A5_adv_pandas.ipynb	   Colab_A11_heatmap.ipynb


In [5]:
# Create the database and save it in a file called store.db
gdrive='drive/MyDrive/cs/cs133/'
Path(gdrive+'store.db').touch()

conn = sqlite3.connect(gdrive+'store.db') # creates a connection to the database
c = conn.cursor() # the cursor is what we'll actually use

In [6]:
# # Delete all rows from the tables
# c.execute('DELETE FROM product')
# c.execute('DELETE FROM customer')
# c.execute('DELETE FROM purchase_order')
# c.execute('DELETE FROM order_item')
# conn.commit()

In [7]:
# Create the the four tables
c.execute('''CREATE TABLE IF NOT EXISTS product (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL
)''')
c.execute('''CREATE TABLE IF NOT EXISTS customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL
)''')
c.execute('''CREATE TABLE IF NOT EXISTS purchase_order (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    date TEXT NOT NULL CHECK (date LIKE '____-__-__'),
    FOREIGN KEY (customer_id)
        REFERENCES customer(customer_id)
            ON DELETE CASCADE
            ON UPDATE NO ACTION
)''')
c.execute('''CREATE TABLE IF NOT EXISTS order_item (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id)
        REFERENCES purchase_order(order_id)
            ON DELETE CASCADE
            ON UPDATE NO ACTION,
    FOREIGN KEY (product_id)
        REFERENCES product(product_id)
            ON DELETE CASCADE
            ON UPDATE NO ACTION
)''')

In [8]:
# 2.3 Load the data in the csv files into the corresponding table.
product = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
product.to_sql('product', conn, if_exists='append', index = False)

customer = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
customer.to_sql('customer', conn, if_exists='append', index = False)

purchase_order = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')
purchase_order.to_sql('purchase_order', conn, if_exists='append', index = False)

order_item = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')
order_item.to_sql('order_item', conn, if_exists='append', index = False)

4

In [9]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
conn.commit()

In [10]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
c.execute('SELECT * FROM product')
print(c.fetchall())
c.execute('SELECT * FROM customer')
print(c.fetchall())
c.execute('SELECT * FROM purchase_order')
print(c.fetchall())
c.execute('SELECT * FROM order_item')
print(c.fetchall())

[(0, 'bicycle', 400.0), (1, 'helmet', 45.0), (2, 'gloves', 23.0), (3, 'chain', 48.0)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]


In [15]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Query 1: customer.name = Wendy Lee
query_1 = '''SELECT customer.customer_id, customer.name, purchase_order.date, product.name, order_item.quantity
          FROM purchase_order
          JOIN customer ON purchase_order.customer_id = customer.customer_id
          JOIN order_item ON purchase_order.order_id = order_item.order_id
          JOIN product ON order_item.product_id = product.product_id
          WHERE customer.name = 'Wendy Lee'
          '''
c.execute(query_1)
print(c.fetchall())

print()

# Query 2: customer.name = Harry Potter
query_2 = '''SELECT customer.customer_id, customer.name, purchase_order.date, product.name, order_item.quantity
          FROM purchase_order
          JOIN customer using (customer_id)
          JOIN order_item using (order_id)
          JOIN product using (product_id)
          WHERE customer.name = 'Harry Potter'
          '''
c.execute(query_2)
print(c.fetchall())

[(0, 'Wendy Lee', '2020-10-19', 'bicycle', 1), (0, 'Wendy Lee', '2020-10-19', 'helmet', 1), (0, 'Wendy Lee', '2020-10-20', 'gloves', 2), (0, 'Wendy Lee', '2020-10-20', 'chain', 1)]

[]


In [ ]:
# Additional tasks
# Appending a new row to one of the tables
# Deleting row(s) with "XXX"

In [16]:
# Appending a new row to one of the tables
new_customer = (4, 'Hung', 'hung@gmail.com')
c.execute("INSERT INTO customer VALUES (?, ?, ?)", new_customer)
conn.commit()

# Verify the insertion
c.execute("SELECT * FROM customer")
print(c.fetchall())

[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu'), (4, 'Hung', 'hung@gmail.com')]


In [17]:
# Delete rows with 'Jason Brown' in the 'name' column of the 'customer' table
c.execute("DELETE FROM customer WHERE name = 'Jason Brown'")
conn.commit()

# Verify the deletion
c.execute("SELECT * FROM customer")
print(c.fetchall())

[(0, 'Wendy Lee', 'wlee@bike.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu'), (4, 'Hung', 'hung@gmail.com')]


In [18]:
conn.close()